# CCTV Timestamp Digit Classification Pipeline

This notebook walks through the complete pipeline to:
1. Extract frames from a CCTV video.
2. Crop timestamp digits from each frame.
3. Manually label the cropped digits.
4. Train a digit classification model (using CNN).
5. Use the trained model to extract time (HH:MM:SS) from new images.

In [1]:
# Step 1: Extract frames from video
import cv2
import os

video_path = 'cctv_video.mp4'  # Path to your video file
output_folder = 'frames'
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)
frame_rate = 1  # Save one frame per second
count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if count % int(cap.get(cv2.CAP_PROP_FPS)) == 0:
        frame_id = count // int(cap.get(cv2.CAP_PROP_FPS))
        cv2.imwrite(f"{output_folder}/frame_{frame_id:05d}.png", frame)
    count += 1
cap.release()

## Step 2: Crop individual digits from timestamp area
Adjust the crop coordinates based on your timestamp location.

In [20]:
import numpy as np

digit_output = 'digits'
os.makedirs(digit_output, exist_ok=True)

# Modify these based on your timestamp layout (6 digits: HHMMSS)
digit_coords = [
    (268, 0, 289, 33),  # hour tens
    (289, 0, 310, 33),  # hour units
    (338, 0, 359, 33),  # minute tens
    (359, 0, 380, 33),  # minute units
    (410, 0, 431, 33),  # second tens
    (431, 0, 452, 33),  # second units
]

frame_files = sorted(os.listdir(output_folder))
for frame_file in frame_files:
    img = cv2.imread(os.path.join(output_folder, frame_file))
    for idx, (x1, y1, x2, y2) in enumerate(digit_coords):
        digit_crop = img[y1:y2, x1:x2]
        digit_path = os.path.join(digit_output, f"{frame_file[:-4]}_digit{idx}.png")
        cv2.imwrite(digit_path, digit_crop)

## Step 3: Manually Label the Cropped Digits
Use a file explorer or labeling tool (like [LabelImg](https://github.com/tzutalin/labelImg)) to label each digit image manually (0-9).

Store them in folders:
`labeled_digits/0`, `labeled_digits/1`, ..., `labeled_digits/9`

## Step 4: Train a CNN Model for Digit Classification

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    'manually_labeled_digits',
    target_size=(50, 30),  # Adjust to match digit size
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    'manually_labeled_digits',
    target_size=(50, 30),
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(50, 30, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, validation_data=val_data, epochs=10)
model.save('digit_classifier_custom.h5')

Found 4938 images belonging to 10 classes.
Found 1229 images belonging to 10 classes.
Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.6668 - loss: 1.0876 - val_accuracy: 0.9561 - val_loss: 0.1853
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9822 - loss: 0.0580 - val_accuracy: 1.0000 - val_loss: 0.0229
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9897 - loss: 0.0242 - val_accuracy: 1.0000 - val_loss: 0.0131
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9845 - loss: 0.0540 - val_accuracy: 0.9780 - val_loss: 0.0420
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9921 - loss: 0.0230 - val_accuracy: 0.9992 - val_loss: 0.0141
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9878 - loss: 0.0281 - val_accuracy: 0.9837 - val_loss: 0.0307
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9946 - loss: 0.0146 - val_accuracy: 0.9992 - val_loss: 0.0122
Epoch 8/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9919 - loss: 0.0199 - val_accuracy: 0.991

## Step 5: Predict Timestamp from New Image Using Trained Model

In [4]:
import os
import shutil
import random

source_dir = 'labeled_digits'
test_dir = 'test_digits'
test_ratio = 0.2  # 20% of images will go to test set

# Create test_digits folders
for i in range(10):
    os.makedirs(os.path.join(test_dir, str(i)), exist_ok=True)

# For each digit class
for digit in range(10):
    digit_path = os.path.join(source_dir, str(digit))
    images = os.listdir(digit_path)
    random.shuffle(images)
    
    test_count = int(len(images) * test_ratio)
    test_images = images[:test_count]
    
    for img in test_images:
        src = os.path.join(digit_path, img)
        dest = os.path.join(test_dir, str(digit), img)
        shutil.copy(src, dest)  # use .copy() if you want to keep originals

print("✅ Dataset split completed! Test images are now in 'test_digits/'.")


FileNotFoundError: [Errno 2] No such file or directory: 'labeled_digits/0'

In [5]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, accuracy_score

# Load the trained model
model = load_model('digit_classifier_custom.h5')

# Set path to test dataset (structured as test_digits/0, test_digits/1, ..., test_digits/9)
test_dir = 'test_digits'  # Change if your folder name differs
img_width, img_height = 30, 50  # Use your model’s input size

X_test = []
y_test = []

# Load images and labels
for label in range(10):
    folder = os.path.join(test_dir, str(label))
    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, (img_width, img_height))
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=-1)  # Add channel dim
        X_test.append(img)
        y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)

# Predict
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Accuracy and detailed report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        29
           1       1.00      1.00      1.00        57
           2       1.00      1.00      1.00        28
           3       1.00      1.00      1.00        24
           4       1.00      1.00      1.00        22
           5       1.00      1.00      1.00        29
           6       1.00      1.00      1.00         7
           7       1.00      1.00      1.00        40
           8       1.00      1.00      1.00        15
           9       1.00      1.00      1.00        10

    accuracy                           1.00       261
   macro avg       1.00      1.00      1.00       261
weighted avg       1.00      1.00      1.00       261



In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the trained digit classification model
model = load_model('digit_classifier_custom.h5')

# Define cropping coordinates for each of the 6 digits (x1, y1, x2, y2)
# Adjust these based on your actual timestamp format
digit_coords = [
    (268, 0, 289, 33),  # hour tens
    (289, 0, 310, 33),  # hour units
    (338, 0, 359, 33),  # minute tens
    (359, 0, 380, 33),  # minute units
    (410, 0, 431, 33),  # second tens
    (431, 0, 452, 33),  # second units
]

def predict_digit(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (30, 50))  # match model input size
    gray = gray.astype('float32') / 255.0
    gray = np.expand_dims(gray, axis=(0, -1))
    pred = model.predict(gray)
    return str(np.argmax(pred))

# Read the timestamp image (replace with your filename)
timestamp_img = cv2.imread('frames/frame_00508.png')

# Crop individual digits
digit_images = [timestamp_img[y1:y2, x1:x2] for (x1, y1, x2, y2) in digit_coords]

# Predict each digit
predicted_digits = [predict_digit(d) for d in digit_images]

# Format as HH:MM:SS
predicted_time = f"{predicted_digits[0]}{predicted_digits[1]}:{predicted_digits[2]}{predicted_digits[3]}:{predicted_digits[4]}{predicted_digits[5]}"
print("Predicted Time:", predicted_time)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Predicted Time: 19:35:15
